# Task 1
### Method (b) - Dense Static Representation

First we import all the libraries we want for this method, and import the training and test csv's.

In [66]:
from gensim.models import Word2Vec
import pandas as pd
import re
import plotly.graph_objects as go
import numpy as np

train_df = pd.read_csv('./Training-dataset.csv')
test_df = pd.read_csv('./Task-1-validation-dataset.csv', header=None)

Now let's clean up the training dataset's synopses to have no punctuation, tags, numbers or special characters.
This will get us an array of words.

In [97]:
clean_sentences = []
count = 0

for w in range(len(train_df['plot_synopsis'])): # For every synopsis

# for w in range(2): # For every synopsis
   

   synopsis = train_df['plot_synopsis'][w].lower()

   count += len(synopsis.split(' '))

   synopsis = re.split(r'\.|\?|\!', synopsis) # split the synopsis into sentences

   for sentence in synopsis:
      sentence = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", sentence) # remove tags
      sentence = re.sub('\'', '', sentence) # change any apostrophe into nothing
      sentence = re.sub('[^a-zA-Z| ]', ' ', sentence) # remove any punctuation
      sentence = re.sub("(\\d|\\W)+", " ", sentence) # remove numbers and special characters
      sentence = sentence.split(' ') # split the sentence by spaces to get the words
      sentence = ([word for word in sentence if len (word) > 0]) # Remove any "empty" characters
      clean_sentences.append(sentence)

print(len(clean_sentences))
print(count)


Call the model. This is where all the hyper parameters are set.

In [93]:
w2v = Word2Vec(clean_sentences, vector_size=100, window=5, min_count=1, workers=16)

Define a cosine similarity function.

This function takes in a model, and 2 words to find the cosine distance between.

We return 0 if the word is OOV (out of vocabulary).

In [94]:
def cosine_similarity(model, word_a, word_b):
    try:
        vector_a = model.wv[word_a]
    except:
        return 0
    try:
        vector_b = model.wv[word_b]
    except:
        return 0

    distance = np.dot(vector_a, vector_b) / (np.linalg.norm(vector_a) * np.linalg.norm(vector_b))
    
    return distance

cosine_similarity(w2v, 'gritty', 'underbelly')

0.8374795

In [95]:
data = [] 
for index, row in test_df.iterrows():
    predicted_similarity = cosine_similarity(w2v, row[1], row[2]) * 10
    data.append([row[0], predicted_similarity])
    print(row[0], row[1], row[2], predicted_similarity)
output_df = pd.DataFrame(data)

1 absorb learn 2.472459226846695
2 absorb withdraw 6.455075740814209
3 achieve accomplish 8.156099915504456
4 achieve try 1.7143160104751587
6 acquire get 3.1269487738609314
7 acquire obtain 8.554601073265076
8 acquire find 3.4299468994140625
11 apple sauce 4.563473463058472
12 apple lemon 4.431261122226715
13 apple sunshine 4.068548083305359
14 area region 7.313403487205505
16 area corner 4.923021197319031
17 arm shoulder 8.5727459192276
20 arm body 5.64653754234314
21 arm vein 2.802685499191284
22 arm knee 7.422813773155212
23 arm bone 5.537571907043457
24 arm neck 8.408842086791992
25 ball cannon 5.3203535079956055
26 ball basket 4.895656108856201
29 bath trick 3.022109270095825
30 bath wife 1.1719601601362228
31 bath balloon 4.3136098980903625
32 bed bedroom 7.414822578430176
33 bed blanket 5.97678542137146
34 bed crib 5.437569618225098
35 bed hospital 4.886257648468018
38 bed chair 7.475957274436951
39 belief opinion 5.994686484336853
40 belief concept 5.350506901741028
41 belief 

In [96]:
from pathlib import Path  
filepath = Path('./Task-1-dense-static-rep.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
print(output_df)
output_df.to_csv(filepath, index=False, header=False)  

       0         1
0      1  2.472459
1      2  6.455076
2      3  8.156100
3      4  1.714316
4      6  3.126949
..   ...       ...
145  177  7.084793
146  178  1.920004
147  179  4.041408
148  181  4.670488
149  182  5.273796

[150 rows x 2 columns]
